## Comparison of GloGEM, PyGEM, and OGGM RGI 11 Runoff Outputs 

This notebook imports and processes GloGEM, PyGEM, and OGGM RGI 11 outputs. Summing glacial runoff by basin, we produce a plot that compares the three models' projected runoff values for each basin by SSP. 

Last Updated: 6 June 2023 | FFW

### Aggregation of GloGEM runoff:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import os

## Generic the filepath to the main data folder
fpath0 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/GloGEM Outputs/CentralEurope/files/'  
fpath1 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/GloGEM Outputs/Runoff-intercomparison/GloGEM/Volume' 

#All of the climate models used
modelnames = ['BCC-CSM2-MR','CAMS-CSM1-0','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0']

SSPpaths = ['ssp126','ssp245','ssp370','ssp585']   #Specifiying the SSP
#SSPs = ['ssp119','ssp126','ssp245','ssp370','ssp585'] #Use a different path as we have all 5 ssps for volume

In [ ]:
all_discharges = [[] for _ in SSPpaths]

for s, SSPpath in enumerate(SSPpaths):
    model_discharges = []
    for modelname in modelnames:
        temp_df = pd.read_csv(fpath0 + modelname + '/' + SSPpaths[s]  + '/' + 'centraleurope_Discharge_r1.dat', sep='\s+', header=None, skiprows=1, index_col=0)
        model_discharges.append(temp_df)
    all_discharges[s] = model_discharges

In [ ]:
all_areas = [[] for _ in SSPpaths]

for s, SSPpath in enumerate(SSPpaths):
    model_areas = []
    for modelname in modelnames:
        temp_df = pd.read_csv(fpath0 + modelname  + '/' + SSPpaths[s]  + '/' + 'centraleurope_Area_r1.dat', sep='\s+', index_col="ID")
        model_areas.append(temp_df)
    all_areas[s] = model_areas

In [ ]:
# Create new index using pandas date_range function
start_date = datetime.date(1980, 1, 1)
end_date = datetime.date(2100, 12, 1)
new_indices = pd.date_range(start_date, end_date, freq='MS').strftime('%Y-%m').tolist()

# Apply new index and datetime conversion
for s, SSPpath_discharges in enumerate(all_discharges):
    for m, discharge_df in enumerate(SSPpath_discharges):
        all_discharges[s][m].columns = new_indices
        all_discharges[s][m].columns = pd.to_datetime(new_indices)

In [ ]:
#expanding area dataset to match year-month dimension
for s in range(len(SSPpaths)):
    for i in range(len(all_areas[s])):
        all_areas[s][i] = all_areas[s][i][all_areas[s][i].columns.repeat(12)]
        
for s, areas in enumerate(all_areas):
    for i, area in enumerate(areas):
        all_areas[s][i].columns = new_indices
        all_areas[s][i].columns = pd.to_datetime(new_indices)

In [ ]:
# We will use our initial area to compute runoff so we fill entire dfs with element 0
# We only use one SSP because the initial areas are all the same -- we save time without looping through all
# We also convert km^2 to m^2

initial_areas = [pd.DataFrame(df.iloc[:, 0].values.repeat(df.shape[1]).reshape(df.shape), index=df.index, columns=df.columns).mul(1e6) for df in all_areas[0]]

In [ ]:
runoff = {s: {m: None for m in modelnames} for s in SSPpaths} # create nested dictionary indexed by model name and ssp
n=0
for s in SSPpaths:
    i=0
    for m in modelnames:
        runoff[s][m] = pd.concat([initial_areas[i] * all_discharges[n][i]], axis=1)
        i+=1
    n+=1

In [ ]:
import itertools
annualrunoff = {s: {m: None for m in modelnames} for s in SSPpaths}
for s, m in itertools.product(SSPpaths, modelnames):
    annualrunoff[s][m] = runoff[s][m].transpose().resample('A').sum()

In [ ]:
import json
def select_glaciers_json(basin='all'):
    '''
    Select glaciers within a basin by MRBID from a json-file,
    which is stored in the data directory.

    Args:
    -----
    basin: str
        String of MRBID or 'all'.

    Returns:
    --------
    If basin is 'all' a list of all relevant glaciers is returned, for
    initiating glacier simulations. If basin is a MRBID the list of glaciers
    within that basin is returned.
    
    Copy of a function written by Erik Holmgren (2022) in holmgren_gha.utils
    '''

    # fpath = './data/rgi_ids_per_basin.json'
    fpath = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/rgi_ids_per_basin.json'  
    with open(fpath) as f:
        basin_dict = json.load(f)

    if basin.lower() != 'all':
        glacier_list = basin_dict[basin]
    else:
        glacier_list = list(itertools.chain.from_iterable(basin_dict.values()))

    return glacier_list

In [ ]:
def sum_basin(basin_RGI_list, runoff_data):
    # Create new list to match our RGI formatting
    new_basin_list = [int(str(x)[-4:]) for x in basin_RGI_list]
    runoff_data = runoff_data.transpose()
    
    #TODO: create list of glaciers within a basin that are not included in GloGEM output
    # Filter new_basin_list to keep only the indexes present in the DataFrame
    new_basin_list = [x for x in new_basin_list if x in runoff_data.index]
    
    # Extract glaciers contained in the list from original df and create a new df
    new_df = runoff_data.loc[new_basin_list].copy()
    
    # Sum the values of the glaciers within the basin
    summed_basin_runoff = new_df.sum()
    #print(summed_basin_runoff)
    
    return summed_basin_runoff

In [ ]:
#Generating the aggregated basin data
alpine_basins = {'RHINE':'6242', 'RHONE':'6243','PO':'6241', 'DANUBE':'6202'} ## GRDC Major River Basin identifiers for the 3 alpine basins we can study
modelnames_glo = ['BCC-CSM2-MR','CAMS-CSM1-0','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0']

Rhone_runoff = {s: {m: None for m in modelnames_glo} for s in SSPpaths} # create nested dictionary indexed by model name and ssp
Rhine_runoff = {s: {m: None for m in modelnames_glo} for s in SSPpaths} 
Po_runoff = {s: {m: None for m in modelnames_glo} for s in SSPpaths}
Danube_runoff = {s: {m: None for m in modelnames_glo} for s in SSPpaths} 

for s in SSPpaths:
    for m in modelnames_glo:
        Rhone_runoff[s][m] = sum_basin(select_glaciers_json(alpine_basins['RHONE']), annualrunoff[s][m]) * 1e-9    #m^3 to km^3
        Rhine_runoff[s][m] = sum_basin(select_glaciers_json(alpine_basins['RHINE']), annualrunoff[s][m]) * 1e-9
        Po_runoff[s][m] = sum_basin(select_glaciers_json(alpine_basins['PO']), annualrunoff[s][m]) * 1e-9
        Danube_runoff[s][m] = sum_basin(select_glaciers_json(alpine_basins['DANUBE']), annualrunoff[s][m]) * 1e-9

In [ ]:
#A quick redefinition that will match PyGEM formatting and make plotting easier later
basins = ['RHINE', 'RHONE', 'PO','DANUBE']
basin_sums_glo = {s: {b: {} for b in basins} for s in SSPpaths}

for s in SSPpaths:
    for m in modelnames_glo:
        basin_sums_glo[s]['RHINE'][m] = Rhine_runoff[s][m]
        basin_sums_glo[s]['RHONE'][m] = Rhone_runoff[s][m]
        basin_sums_glo[s]['PO'][m] = Po_runoff[s][m]
        basin_sums_glo[s]['DANUBE'][m] = Danube_runoff[s][m]

#To calculate multi GCM means and Quartiles we convert to df then calculate across first axis (GCMs)
GCM_mean_glo = {}
GCM_q1_glo = {}
GCM_q3_glo = {}
for SSP in SSPpaths:
    GCM_mean_glo[SSP] = {}
    GCM_q1_glo[SSP] = {}
    GCM_q3_glo[SSP] = {}
    for b in basins:
        GCM_mean_glo[SSP][b] = pd.DataFrame(basin_sums_glo[s][b]).mean(axis=1)
        GCM_q1_glo[SSP][b] = pd.DataFrame(basin_sums_glo[s][b]).quantile(q=0.25, axis=1)
        GCM_q3_glo[SSP][b] = pd.DataFrame(basin_sums_glo[s][b]).quantile(q=0.75, axis=1)

### Aggregation of PyGEM runoff:

In [ ]:
import xarray as xr

#All of the climate models used
modelnames_py = ['BCC-CSM2-MR','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM']

SSPs = ['ssp126','ssp245','ssp370','ssp585'] #List of all SSPs in PyGEM
which_ssp = SSPs[0]

alpine_basins = {'RHINE': '6242',
                 'RHONE': '6243',
                 'PO': '6241',
                 'DANUBE':'6202'} ## GRDC Major River Basin identifiers for the 4 alpine basins we can study

test_basin = alpine_basins['RHONE'] 

#Generic filepath to navigate to Drive folder 
fpathPy = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/11'

In [ ]:
basin_gls = {}
for basin, code in alpine_basins.items():
    basin_gls[basin] = select_glaciers_json(code)

In [ ]:
#Importing all runoff data, taking annual sum, and converting m^3 to km^3
import glob   #use glob to group files by filename similarities (in this case, SSP)

rf_ds = {}
annual_rf_ds = {}
for s, SSP in enumerate(SSPs):
    fpath1 = '/R11_runoff_monthly_c2_ba1_1set_2000_2100-{}'.format(SSP)
    file_pattern = f'{fpathPy + fpath1}*.nc'
    file_list = glob.glob(file_pattern)
    
    datasets = []  # Create an empty list for each SSP
    if file_list:
        for file in file_list:
            with xr.open_dataset(file) as ds:
                ds = ds.glac_runoff_monthly.load()
                datasets.append(ds)
    
        combined_ds = xr.concat(datasets, dim='glacier')  # Concatenate the datasets
        rf_ds[SSP] = combined_ds
        annual_rf_ds[SSP] = rf_ds[SSP].resample(time='A').sum() * 1e-9

In [ ]:
# Sorting into basins
basin_datasets = {}
for basin, glacier_list in basin_gls.items():
    ## loop over them all, drop the irrelevant IDs, and concatenate the result
    basin_datasets[basin] = {}
    for s, SSP in enumerate(SSPs):
        ds_list = []
        try:
            ds_filtered = annual_rf_ds[SSP].where(annual_rf_ds[SSP].RGIId.isin(glacier_list), drop=True)
            #print(ds_filtered)
            ds_list.append(ds_filtered)
        except ValueError: ## happens if there are no glaciers from this batch in the selected region
            continue
        basin_datasets[basin][SSP] = xr.concat(ds_list, dim='glacier')

In [ ]:
#Flipping indexing (to match other models) and summing basins
basin_sums_py = {}
for s, SSP in enumerate(SSPs):        
    basin_sums_py[SSP] = {}
    for basin, glacier_list in basin_gls.items():
        basin_sums_py[SSP][basin] = basin_datasets[basin][SSP].sum(dim='glacier')

In [ ]:
#Compute multi GCM means and quartiles
GCM_mean_py = {}
GCM_q1_py = {}
GCM_q3_py = {}
for s, SSP in enumerate(SSPs):
    which_ssp = SSPs[s]
    GCM_mean_py[which_ssp] = {}
    GCM_q1_py[which_ssp] = {}
    GCM_q3_py[which_ssp] = {}
    for basin in basins:
        GCM_mean_py[which_ssp][basin] = basin_sums_py[which_ssp][basin].mean(dim = 'model')
        GCM_q1_py[which_ssp][basin] = basin_sums_py[which_ssp][basin].quantile(q = 0.25, dim = 'model')
        GCM_q3_py[which_ssp][basin] = basin_sums_py[which_ssp][basin].quantile(q = 0.75, dim = 'model')

In [ ]:
from cycler import cycler
import matplotlib.patches as mpatches

scenarios = ['ssp126','ssp245','ssp370','ssp585']
basins = ['RHINE', 'RHONE', 'PO','DANUBE']

yrs_glo = np.arange(1980,2101)
yrs_glo_dt = pd.to_datetime([str(y)for y in yrs_glo])

colors_glo =  plt.colormaps['Greens']
line_colors_glo = colors_glo(np.linspace(0.2, 0.6, num = 12))
glo_cycler = cycler(color = line_colors_glo)

colors_py =  plt.colormaps['Purples']
line_colors_py = colors_py(np.linspace(0.2, 0.6,num = 12))
py_cycler = cycler(color = line_colors_py)

ssp_label_pos = [1.9, 4.73, 1.45, 2.0]

#Plotting all data
fig, axs = plt.subplots(len(scenarios), 4, figsize=(12, 10), sharex=True)
for s, SSP in enumerate(scenarios):
    which_ssp = SSPs[s]
    for b, basin in enumerate(basins):
        for m in modelnames_glo:
            axs[s, b].plot(yrs_glo_dt, basin_sums_glo[which_ssp][basin][m], color=axs[s, b].set_prop_cycle(glo_cycler), alpha = 0.25)
        axs[s,b].plot(yrs_glo_dt, GCM_mean_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.9)
        axs[s,b].plot(yrs_glo_dt, GCM_q1_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.4)
        axs[s,b].plot(yrs_glo_dt, GCM_q3_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.4)
        axs[s,b].fill_between(yrs_glo_dt, GCM_q1_glo[which_ssp][basin], GCM_q3_glo[which_ssp][basin], color = 'green')
        axs[s, b].set(xlim=(pd.to_datetime('2000-01-01'), pd.to_datetime('2100-01-01')))

        basin_sums_py[which_ssp][basin].plot(hue='model', ax=axs[s, b], color=axs[s, b].set_prop_cycle(py_cycler), alpha = 0.25, add_legend=False)
        GCM_mean_py[which_ssp][basin].plot(hue='model', ax=axs[s, b], color = 'purple', linewidth = 0.9, add_legend=False)
        GCM_q1_py[which_ssp][basin].plot(hue='model', ax=axs[s, b], color = 'purple', linewidth = 0.4, add_legend=False)
        GCM_q3_py[which_ssp][basin].plot(hue='model', ax=axs[s, b], color = 'purple', linewidth = 0.4, add_legend=False)
        axs[s,b].fill_between(yrs_glo_dt[20::], GCM_q1_py[which_ssp][basin], GCM_q3_py[which_ssp][basin], color = 'Purple')
        axs[s,b].set(title = '')

        #Setting x and y labels and making y limits uniform within basins
        if s == 3:
            for sub_b in range(4):  # Use a different variable name for the inner loop
                axs[s, sub_b].set_xlabel('Year')
                axs[s, sub_b].set_xticks([pd.to_datetime('2020'),pd.to_datetime('2040'), pd.to_datetime('2060'), pd.to_datetime('2080')], [2020, 2040, 2060, 2080])
        else:
            axs[s, b].set_xlabel(None)
            
        if b == 0:
            for sub_s in range(4):  # Use a different variable name for the inner loop
                axs[sub_s, b].set(ylim = (0.4,2.1))
                if sub_s == 2:
                    axs[sub_s, b].set_ylabel(r'Multi-GCM Runoff Mean and Quartiles $[km^3]$', y=1.1)
            else:
                axs[s, b].set_ylabel(None)
            
        if b == 1:
            for sub_s in range(4):  # Use a different variable name for the inner loop
                axs[sub_s, b].set(ylim = (0.8,5.2))
                axs[s, b].set_ylabel(None)
        if b == 2:
            for sub_s in range(4):  # Use a different variable name for the inner loop
                axs[sub_s, b].set_yticks([0.5, 1.0, 1.5]) 
                axs[sub_s, b].set(ylim = (0.3,1.6))
                axs[s, b].set_ylabel(None)
        if b == 3:
            for sub_s in range(4):  # Use a different variable name for the inner loop
                axs[sub_s, b].set(ylim = (0.4,2.2))
                axs[s, b].set_ylabel(None)

        axs[s, b].text(pd.to_datetime('2070-01-01'), ssp_label_pos[b], SSP)

green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
purple_patch = mpatches.Patch(color='purple', label='PyGEM')       
axs[0,0].legend(handles=[green_patch, purple_patch], bbox_to_anchor=(2.8, 1.48), ncol=2)

#plt.subplots_adjust(hspace = 0.5)
plt.suptitle('GloGEM and PyGEM Runoff Projections for Major Central European River Basins')
plt.title('Rhine River Basin                 Rhone River Basin                    Po River Basin                   Danube River Basin', x=-1.28, y=4.72)

### Introducing OGGM Runoff

In [ ]:
#All of the climate models used
modelnames_OG = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'CMCC-CM2-SR5','EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8',
               'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM', 'TaiESM1']

# CMCC-CM2-SR5 & TaiESM1 only hold values for ssp585––this is model list without those GCMS
modelnames_OG_trimmed = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8',
               'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

#Generic filepath to navigate to Drive folder 
fpathOG = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/OGGM Outputs/'

In [ ]:
#Importing all runoff data, OGGM is grouped by basin
rf_ds = {}
for basin, code in alpine_basins.items():
    fpath_basin = 'lschuster/{}'.format(code)
    #print(f'{fpathOG + fpath_basin} (1)/*.nc')
    with xr.open_mfdataset(f'{fpathOG + fpath_basin} (1)/*.nc') as ds:
        ds = ds.runoff.load()
    rf_ds[basin] = ds

In [ ]:
#Summing individual glacier runoff into basin totals and converting kg to km^3
basin_rf_OG = {}
for basin, code in alpine_basins.items():
    basin_rf_OG[basin] = rf_ds[basin].sum(dim = 'rgi_id') * 1e-12

In [ ]:
#creating dict of GloPy format
basins = ['RHINE', 'RHONE', 'PO', 'DANUBE']
basin_sums_OG = {}
for s, SSP in enumerate(SSPs):
    which_ssp = SSPs[s]
    basin_sums_OG[which_ssp] = {}
    for b, basin in enumerate(basins):
        basin_sums_OG[which_ssp][basin] = basin_rf_OG[basin].sel(scenario = which_ssp)

In [ ]:
#Plotting it looks like some GCMs contain all-zero values, like:
basin_sums_OG['ssp126']['DANUBE'][14]

In [ ]:
#Removing these GCMs for ALL SSPs--doing even 585 as these two are not included...
#... in Glo or Py so not only makeds OOGM easier but maintains GCM consistency in analysis
trimmed_basin_sums_OG = {}
for s, SSP in enumerate(SSPs):
    which_ssp = SSPs[s]
    trimmed_basin_sums_OG[which_ssp] = {}
    for b, basin in enumerate(basins):
        trimmed_basin_sums_OG[which_ssp][basin] = xr.concat([basin_sums_OG[which_ssp][basin][0:4], basin_sums_OG[which_ssp][basin][5:-1]], dim='gcm')

In [ ]:
#Compute multi GCM means and quartiles for OGGM
GCM_mean_OG = {}
GCM_q1_OG = {}
GCM_q3_OG = {}
for s, SSP in enumerate(SSPs):
    which_ssp = SSPs[s]
    GCM_mean_OG[which_ssp] = {}
    GCM_q1_OG[which_ssp] = {}
    GCM_q3_OG[which_ssp] = {}
    for basin in basins:
        GCM_mean_OG[which_ssp][basin] = trimmed_basin_sums_OG[which_ssp][basin].mean(dim = 'gcm')
        GCM_q1_OG[which_ssp][basin] = trimmed_basin_sums_OG[which_ssp][basin].quantile(q = 0.25, dim = 'gcm')
        GCM_q3_OG[which_ssp][basin] = trimmed_basin_sums_OG[which_ssp][basin].quantile(q = 0.75, dim = 'gcm')

In [ ]:
#Plot setup
from cycler import cycler
import matplotlib.patches as mpatches

scenarios = ['ssp126','ssp245','ssp370','ssp585']
basins = ['RHINE', 'RHONE', 'PO','DANUBE']
basinstext = ['Rhine', 'Rhone', 'Po','Danube']

yrs_glo = np.arange(1980,2101)
yrs_glo_dt = pd.to_datetime([str(y)for y in yrs_glo])

colors_glo =  plt.colormaps['Greens']
line_colors_glo = colors_glo(np.linspace(0.2, 0.6, num = 12))
glo_cycler = cycler(color = line_colors_glo)

colors_py =  plt.colormaps['Purples']
line_colors_py = colors_py(np.linspace(0.2, 0.6,num = 12))
py_cycler = cycler(color = line_colors_py)

colors_OG =  plt.colormaps['Blues']
line_colors_OG = colors_OG(np.linspace(0.2, 0.6,num = 12))
OG_cycler = cycler(color = line_colors_OG)

In [ ]:
#Correct for GloGEM hydro vs regular year indexing???
start_date = datetime.date(1979, 10, 1)
end_date = datetime.date(2100, 9, 1)
yrs = pd.date_range(start_date, end_date, freq='MS')

In [ ]:
#Same plot as above with OGGM data added
#Also flipping axes so that l to r reads as one basin with increasing scenario severity 
#Plotting all data
fig, axs = plt.subplots(len(scenarios), 4, figsize=(12, 10), sharex=True)
for s, SSP in enumerate(scenarios):
    which_ssp = SSPs[s]
    for b, basin in enumerate(basins):

        #OG won't plot with built-in ds.plot()
        #Trim last value as it goes to zero
        for m, model in enumerate(modelnames_OG_trimmed):
            axs[b,s].plot(yrs_glo_dt[20:-1], trimmed_basin_sums_OG[which_ssp][basin][:,0:-1].sel(gcm = modelnames_OG_trimmed[m]), color = 'dodgerblue', alpha = 0.15)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_mean_OG[which_ssp][basin][0:-1], color = 'royalblue', linewidth = 0.9)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_q1_OG[which_ssp][basin][0:-1], color = 'royalblue', linewidth = 0.4)
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_q3_OG[which_ssp][basin][0:-1], color = 'royalblue', linewidth = 0.4)
        axs[b,s].fill_between(yrs_glo_dt[20:-1], GCM_q1_OG[which_ssp][basin][0:-1], GCM_q3_OG[which_ssp][basin][0:-1], color = 'dodgerblue', alpha = 0.5)

        #Trim first value as it is incomplete hydrological year
        for m in modelnames_glo:
            axs[b, s].plot(yrs_glo_dt, basin_sums_glo[which_ssp][basin][m], color=axs[b, s].set_prop_cycle(glo_cycler), alpha = 0.25)
        axs[b,s].plot(yrs_glo_dt, GCM_mean_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.9)
        axs[b,s].plot(yrs_glo_dt, GCM_q1_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.4)
        axs[b,s].plot(yrs_glo_dt, GCM_q3_glo[which_ssp][basin], color = 'darkgreen', linewidth = 0.4)
        axs[b,s].fill_between(yrs_glo_dt, GCM_q1_glo[which_ssp][basin], GCM_q3_glo[which_ssp][basin], color = 'green')
        axs[b, s].set(xlim=(pd.to_datetime('2000-01-01'), pd.to_datetime('2100-01-01')))

        basin_sums_py[which_ssp][basin].plot(hue='model', ax=axs[b, s], color=axs[b, s].set_prop_cycle(py_cycler), alpha = 0.25, add_legend=False)
        GCM_mean_py[which_ssp][basin].plot(hue='model', ax=axs[b, s], color = 'purple', linewidth = 0.9, add_legend=False)
        GCM_q1_py[which_ssp][basin].plot(hue='model', ax=axs[b, s], color = 'purple', linewidth = 0.4, add_legend=False)
        GCM_q3_py[which_ssp][basin].plot(hue='model', ax=axs[b, s], color = 'purple', linewidth = 0.4, add_legend=False)
        axs[b,s].fill_between(yrs_glo_dt[20::], GCM_q1_py[which_ssp][basin], GCM_q3_py[which_ssp][basin], color = 'Purple')
        axs[b,s].set(title = '')

        #Make mean more clear for RHONE, which overlaps significantly w Glo
        axs[b,s].plot(yrs_glo_dt[20:-1], GCM_mean_OG[which_ssp][basin][0:-1], color = 'royalblue', linewidth = 0.9)

        #Setting x and y labels and making y limits uniform within basins
        if b == 3:
            for sub_s in range(4):  # Use a different variable name for the inner loop
                axs[b, sub_s].set_xlabel('Year')
                axs[b, sub_s].set_xticks([pd.to_datetime('2020'),pd.to_datetime('2040'), pd.to_datetime('2060'), pd.to_datetime('2080')], [2020, 2040, 2060, 2080])
        else:
            axs[b, s].set_xlabel(None) 
        if b == 0:
            for sub_s in range(4):  # Use a different variable name for the inner loop
                axs[b, sub_s].set_yticks([0.5, 1.0, 1.5, 2.0, 2.5]) 
                axs[b, sub_s].set(ylim = (0.4,2.6))
        if b == 1:
            for sub_s in range(4): 
                y_ticks = [1.0, 2.0, 3.0, 4.0, 5.0]
                y_tick_labels = [f'{tick:.1f}' for tick in y_ticks] 
                axs[b, sub_s].set_yticks(y_ticks)
                axs[b, sub_s].set_yticklabels(y_tick_labels)
                axs[b, sub_s].set(ylim = (0.8,5.5))
        if b == 2:                                                                   #Setting y-limits for each basin
            for sub_s in range(4):  
                axs[b, sub_s].set_yticks([0.5, 1.0, 1.5, 2.0]) 
                axs[b, sub_s].set(ylim = (0.3,2.1))
        if b == 3:
            for sub_s in range(4):  
                axs[b, sub_s].set(ylim = (0.4,2.6))
        if s == 0:                                                                    #Setting basin labels
            for sub_b in range(4):
                axs[sub_b,s].set_ylabel(basinstext[sub_b]+ r' Runoff $[km^3]$')
        if s != 0:
            axs[b, s].set_ylabel(None)

green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
purple_patch = mpatches.Patch(color='purple', label='PyGEM') 
blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
axs[0,0].legend(handles=[green_patch, purple_patch, blue_patch], bbox_to_anchor=(3.15, 1.46), ncol=3)

plt.suptitle('GloGEM, PyGEM, and OGGM Runoff Projections for Major Central European River Basins', x=0.52)
plt.title('SSP 126                                 SSP 245                                SSP 370                                 SSP 585', x=-1.3, y=4.66) 